In [24]:
import errant
import pandas as pd
import difflib

In [86]:
def get_diff(a,b):
    if len(a)>len(b): 
        res=''.join(a.split(b))             
    else: 
        res=''.join(b.split(a))             
    return res.strip()

In [87]:
punctuation = ['.',',','!','?','..','...','-']

In [88]:
def error_type(s1,s2):
    annotator = errant.load('en')
    orig = annotator.parse(s1)
    cor = annotator.parse(s2)
    edits = annotator.annotate(orig, cor)
    for e in edits:
        if 'DET' in e.type:
            return 'Articles'
        elif 'PREP' in e.type or 'PART' in e.type:
            return 'Preposition'
        elif 'PUNCT' in e.type or (e.o_str!='' and e.c_str!='' and get_diff(e.o_str,e.c_str) in punctuation) or ('-' in e.o_str or '-' in e.c_str):
            return 'Punctuation'
        elif 'VERB' in e.type:
            if 'SVA' in e.type:
                return 'Subject Verb Agreement'
            else:
                return 'Verb Form'
        elif 'NOUN' in e.type or 'ADJ' in e.type or 'MORPH' in e.type:
            return 'Word Form'
        else:
            return 'Other'

In [98]:
def get_action(s1,s2):
    annotator = errant.load('en')
    orig = annotator.parse(s1)
    cor = annotator.parse(s2)
    edits = annotator.annotate(orig, cor)
    for e in edits:
        if 'R:' in e.type:
            return 'replace'
        elif 'M:' in e.type:
            return 'insert'
        elif 'U:' in e.type:
            return 'remove'
            
            

In [107]:
def get_expla(s1,s2):
    annotator = errant.load('en')
    orig = annotator.parse(s1)
    cor = annotator.parse(s2)
    edits = annotator.annotate(orig, cor)
    if error_type(s1,s2)=='Verb Form':
        for e in edits:
            if 'TENSE' in e.type:
                return 'Verb Tense Error'
            elif 'FORM' in e.type:
                return 'Verb Form error'
            else:
                return 'Other verb error'
    elif error_type(s1,s2)=='Word Form':
        for e in edits:
            if 'NUM' in e.type:
                return 'Noun Number error'
            elif 'ADJ' in e.type:
                return 'Adjective error'
            elif 'MORPH' in e.type:
                return 'Morphology error'
            else:
                return 'Other word form error'
            
        
        

In [9]:
data = pd.read_csv('ErrorData.csv')

In [99]:
data['give_action'] = data.apply(lambda x: get_action(x.question, x.solution),axis=1)

In [96]:
data['give_category'] = data.apply(lambda x: error_type(x.question, x.solution),axis=1)

In [100]:
data.head(30)

,questionid,category,round,question,solution,explanation,type,action,give_category,give_action
0,1,Articles,1,These devices generate huge amount of data tha...,These devices generate a huge amount of data t...,Article required,C,insert,Articles,insert
1,1,Articles,1,Fashion items make up large portion of e-comme...,Fashion items make up a large portion of e-com...,"The word ""portion"" is a single, countable noun...",SP,insert,Articles,insert
2,1,Articles,2,"Firstly, increasing number of people are reviv...","Firstly, an increasing number of people are re...",Article required,C,insert,Articles,insert
3,1,Articles,3,"In the recent years, the rise of technology in...","In recent years, the rise of technology in the...",No article required,C,remove,Articles,remove
4,1,Articles,4,"The Sugar Detox Diet, which aims to promote a ...","The Sugar Detox Diet, which aims to promote he...",No article required,C,remove,Articles,remove
5,1,Preposition,1,A study by ABI Research (2017) forecasted that...,A study by ABI Research (2017) forecasted that...,Idiomatic expressions: will grow by and will g...,C,insert,Preposition,insert
6,1,Preposition,1,One method to measure brand positioning would ...,One method to measure brand positioning would ...,"Modals or modal auxiliary verbs (can, could, m...",SP,insert,Verb Form,insert
7,1,Preposition,2,I have researched on recent health food trends.,I have researched recent health food trends.,No preposition required after research as a verb,C,remove,Preposition,remove
8,1,Preposition,3,Board games help people to build up social ski...,Board games help people to build social skills...,No preposition required because the direction ...,C,remove,Preposition,remove
9,1,Preposition,4,The cost of a mobile phone can amount from $1000.,The cost of a mobile phone can amount to $1000.,"Amount to means up to an amount, as in this case.",C,replace,Preposition,replace


In [110]:
s1 = 'It is forecasted that AR users will increase by another 35% over the next 5 years and millennials play a large part in that (Madigan, 2016).'
s2 = 'It is forecasted that AR users will increase by another 35% over the next 5 years and millennials will play a large part in that (Madigan, 2016).'



In [111]:
get_expla(s1,s2)

'Verb Tense Error'

In [105]:
s3 = 'The rising incidences of stroke among young people is an issue of concern.'
s4 = 'The rising incidence of stroke among young people is an issue of concern.'

In [109]:
get_expla(s3,s4)

'Noun Number error'